# Control based on León paper

A. E. Leon and J. A. Solsona, "Performance Improvement of Full-Converter Wind Turbines Under Distorted Conditions," in IEEE Transactions on Sustainable Energy, vol. 4, no. 3, pp. 652-660, July 2013, doi: 10.1109/TSTE.2013.2239317.

In [1]:
%matplotlib widget

In [2]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
import scipy.signal as sctrl
import pydae.ssa as ssa
from sympy.physics.quantum import TensorProduct

### Plant model

In [22]:
Δt = 50e-6
R_t =  0.039269908169872414
L_t =  0.00125
C_m =  4e-06
G_d =  1.0
R_s =  0.039269908169872414
L_s =  0.00125

A = np.array([
    [-R_t/L_t,       0,   -1/L_t,        0,        0,        0],
    [       0,-R_t/L_t,        0,   -1/L_t,        0,        0],
    [   1/C_m,       0, -G_d/C_m,        0,   -1/C_m,        0],
    [       0,   1/C_m,        0, -G_d/C_m,        0,   -1/C_m],
    [       0,       0,    1/L_s,        0, -R_s/L_s,        0],
    [       0,       0,        0,    1/L_s,        0, -R_s/L_s],
    ])

B = np.array([
    [ 1/L_t,        0],
    [       0,  1/L_t],
    [       0,      0],
    [       0,      0],
    [       0,      0],
    [       0,      0],
    ])

B_g = np.array([
    [       0,      0],
    [       0,      0],
    [       0,      0],
    [       0,      0],
    [-1/L_s,        0],
    [       0, -1/L_s],
    ])

C_c = np.array([
    [ 0, 0, 0, 0, 1, 0],
    [ 0, 0, 0, 0, 0, 1],
    ])

D_c = np.array([
    [ 0, 0],
    [ 0, 0],
    ])

C_o = np.array([
    [ 0, 0, 1, 0, 0, 0],
    [ 0, 0, 0, 1, 0, 0],
    [ 0, 0, 0, 0, 1, 0],
    [ 0, 0, 0, 0, 0, 1],
    ])

D_o = np.array([
    [ 0, 0],
    [ 0, 0],
    ])

# plant discretization
A_d,B_d,C_d,D_d,Dt = sctrl.cont2discrete((A,B,C_c,D_c),Δt,method='zoh')
A_,B_gd,C_,D_,Dt   = sctrl.cont2discrete((A,B_g,C_c,D_c),Δt,method='zoh')
A_,B_,C_o,D_o,Dt   = sctrl.cont2discrete((A,B,C_o,D_o),Δt,method='zoh')


## Park aplication

In [23]:
omega_g,t_k,Δt_sym = sym.symbols('omega_g,t_k,Δt_sym')
theta_k = omega_g*t_k
theta_kp1 = omega_g*(t_k + Δt_sym)

P_k =  sym.Matrix([
                [   sym.cos(theta_k), -sym.sin(theta_k)],   
                [  -sym.sin(theta_k), -sym.cos(theta_k)],    
               ])

P_kp1 =  sym.Matrix([
                [   sym.cos(theta_kp1), -sym.sin(theta_kp1)],   
                [  -sym.sin(theta_kp1), -sym.cos(theta_kp1)],    
               ])

W_sym = sym.simplify(P_kp1 @ P_k.inv())

m2 = sym.Matrix([[1,0,0],[0,1,0],[0,0,1]])
P_kp1_3 = TensorProduct(m2, P_kp1)
P_ki_3  = TensorProduct(m2, P_k.inv())

A_b_sym   = sym.simplify(P_kp1_3 @ A_d  @ P_ki_3 )
B_b_sym   = sym.simplify(P_kp1_3 @ B_d  @ P_k.inv() )
B_g_bsym =  sym.simplify(P_kp1_3 @ B_gd @ P_k.inv() )

In [24]:
A_b_eval = sym.lambdify([omega_g,Δt_sym], A_b_sym)
B_b_eval = sym.lambdify([omega_g,Δt_sym], B_b_sym)
B_g_b_eval = sym.lambdify([omega_g,Δt_sym], B_g_bsym)
W_eval  = sym.lambdify([omega_g,Δt_sym], W_sym) 

In [25]:
omega_b = 2*np.pi*50
A_b = A_b_eval(omega_b,Δt)
B_b = B_b_eval(omega_b,Δt)
B_g_b = B_g_b_eval(omega_b,Δt)
W =  W_eval(omega_b,Δ𝑡)

## Controller

In [26]:
# Controller ##################################################################################
N_x_c,N_u_d = B_b.shape
N_z_c,N_x_c = C_c.shape


O_ux = np.zeros((N_u_d,N_x_c))
O_xu = np.zeros((N_x_c,N_u_d))
O_uu = np.zeros((N_u_d,N_u_d))
I_uu = np.eye(N_u_d)


# discretized plant:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δz_c = C_c*Δx_d + D_c*Δu_d

# dinamic extension:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δx_i = Δx_i + Δt*(Δz_c-Δz_c_ref) = Δx_i + Δt*C_c*Δx_d - Dt*Δz_c_ref
# Δz_c = z_c - z_c_0
# Δz_c_ref = z_c_ref - z_c_0
# (Δz_c-Δz_c_ref) = z_c - z_c_ref


A_e = np.block([
                [    A_b,    B_b, O_xu],    # Δx_d
                [   O_ux,   O_uu, O_uu],    # Δx_r
                [ Δt*C_d,   O_uu, I_uu],    # Δx_i    
               ])

B_e = np.block([
                [   O_xu],
                [      W],
                [   O_uu],    
               ])



# weighting matrices
Q_c = np.eye(A_e.shape[0])
Q_c[-1,-1] = 1e8
Q_c[-2,-2] = 1e8

R_c = np.eye(B_e.shape[1])*10

K_c,S_c,E_c = ssa.dlqr(A_e,B_e,Q_c,R_c)

E_cont = np.log(E_c)/Δt

## Observer

In [27]:
N_z_o = C_o.shape[0]

Q_o = np.eye(A_d.shape[0])

R_o = np.diag([1]*N_z_o)
K_o_T,S_o,E_o = ssa.dlqr(A_d.T,C_o.T,Q_o,R_o)
K_o = K_o_T.T


print('damp_ctrl',-E_c.real/np.abs(E_c))
print('damp_obs',-E_o.real/np.abs(E_o))

damp_ctrl [-0.99989883 -0.99989883 -0.99881325 -0.99881325 -0.9995029  -0.9995029
 -0.99987663 -0.99987663 -1.         -1.        ]
damp_obs [-1. -1. -1. -1. -1. -1.]


## Simulink

In [28]:
# Control without observer Du_r = -K_c*Dx_e
x_d_1,x_d_2,x_d_3,x_d_4,x_d_5,x_d_6 = sym.symbols('Dx_d_1,Dx_d_2,Dx_d_3,Dx_d_4,Dx_d_5,Dx_d_6')
x_r_1,x_r_2 = sym.symbols('Dx_r_1,Dx_r_2')
x_i_1,x_i_2 = sym.symbols('Dx_i_1,Dx_i_2')

x_e = sym.Matrix([x_d_1,x_d_2,x_d_3,x_d_4,x_d_5,x_d_6,x_r_1,x_r_2,x_i_1,x_i_2])
u_r = -K_c * x_e

u_r_d = str(sym.N(u_r[0],8))
u_r_q = str(sym.N(u_r[1],8))

print(f'Du_r_1 = {u_r_d};')
print(f'Du_r_2 = {u_r_q};')

print('\nWarning: Control output is v_t_dq!!')

Du_r_1 = -1.8262175*Dx_d_1 + 0.019754652*Dx_d_2 - 0.0048070784*Dx_d_3 + 0.0010494948*Dx_d_4 - 3.4543535*Dx_d_5 + 0.39761572*Dx_d_6 - 2715.7977*Dx_i_1 + 1040.9568*Dx_i_2 - 0.071871552*Dx_r_1 + 0.00052148736*Dx_r_2;
Du_r_2 = -0.019754652*Dx_d_1 - 1.8262175*Dx_d_2 - 0.0010494948*Dx_d_3 - 0.0048070784*Dx_d_4 - 0.39761572*Dx_d_5 - 3.4543535*Dx_d_6 - 1040.9568*Dx_i_1 - 2715.7977*Dx_i_2 - 0.00052148736*Dx_r_1 - 0.071871552*Dx_r_2;



In [29]:
it_ini = 4
Δx_o = sym.Matrix([sym.Symbol(f'xD[{it+it_ini}]') for it in range(6)])
Δz_o = sym.Matrix([sym.Symbol(item) - sym.Symbol(item+'_0') for item in ['v_md', 'v_mq', 'i_sd', 'i_sq']])
Δx_r = sym.Matrix([sym.Symbol(f'Dx_r_{it+1}') for it in range(2)])
Δu_pert = sym.Matrix([sym.Symbol(f'Du_pert_{it+1}') for it in range(2)])

Δx_o_kp1 = A_d @ Δx_o + B_b@(Δx_r) +  K_o @ (Δz_o - C_o @ Δx_o) # + B_pert@Δu_pert  +  K_o @ (Δz_o - C_o @ Δx_o - D_o @ (Δx_r))

for it in range(6):
    print(f'xD[{it+it_ini}] = {Δx_o_kp1[it]};')

xD[4] = 0.0392952872921423*Dx_r_1 + 0.000617299701089028*Dx_r_2 + 0.167670572903802*i_sd - 0.167670572903802*i_sd_0 + 0.427599429013122*v_md - 0.427599429013122*v_md_0 + 0.962811636697318*xD[4] - 0.430586049216921*xD[6] - 0.132051772873075*xD[8];
xD[5] = -0.000617299701089028*Dx_r_1 + 0.0392952872921423*Dx_r_2 + 0.167670572903802*i_sq - 0.167670572903802*i_sq_0 + 0.427599429013122*v_mq - 0.427599429013122*v_mq_0 + 0.962811636697318*xD[5] - 0.430586049216921*xD[7] - 0.132051772873075*xD[9];
xD[6] = 0.0356406531431186*Dx_r_1 + 0.000559888120127431*Dx_r_2 - 0.425448653071038*i_sd + 0.425448653071038*i_sd_0 + 0.443158057967852*v_md - 0.443158057967852*v_md_0 + 0.933318813687106*xD[4] - 0.449166750687225*xD[6] - 0.507870160616068*xD[8];
xD[7] = -0.000559888120127431*Dx_r_1 + 0.0356406531431186*Dx_r_2 - 0.425448653071038*i_sq + 0.425448653071038*i_sq_0 + 0.443158057967853*v_mq - 0.443158057967853*v_mq_0 + 0.933318813687106*xD[5] - 0.449166750687225*xD[7] - 0.507870160616068*xD[9];
xD[8] = 0.

In [30]:
#eta_dq[0] = Du_r_1*2/v_dc[0] + (v_sd)*2/v_dc[0] ; 
#eta_dq[1] = Du_r_2*2/v_dc[0] + (v_sq)*2/v_dc[0] ;
#Du_r_1 = eta_dq[0]*v_dc[0]/2
#Du_r_2 = eta_dq[1]*v_dc[0]/2

Du_r_1,Du_r_2 = sym.symbols('Du_r_1,Du_r_2')
Du_r = sym.Matrix([Du_r_1,Du_r_2 ])

Dx_r = W@Du_r

Dx_r_1 = str(sym.N(Dx_r[0],8))
Dx_r_1 = str(sym.N(Dx_r[1],8))

print(f'Dx_r_1 = {Dx_r_1};')
print(f'Dx_r_2 = {Dx_r_1};')

Dx_r_1 = -0.015707317*Du_r_1 + 0.99987663*Du_r_2;
Dx_r_2 = -0.015707317*Du_r_1 + 0.99987663*Du_r_2;


In [32]:
# Control with observer Du_r = -K_c*Dx_o
x_d_1,x_d_2,x_d_3,x_d_4,x_d_5,x_d_6 = sym.symbols('xD[4],xD[5],xD[6],xD[7],xD[8],xD[9]')
x_r_1,x_r_2 = sym.symbols('Dx_r_1,Dx_r_2')
x_i_1,x_i_2 = sym.symbols('Dx_i_1,Dx_i_2')

x_e = sym.Matrix([x_d_1,x_d_2,x_d_3,x_d_4,x_d_5,x_d_6,x_r_1,x_r_2,x_i_1,x_i_2])
u_r = -K_c * x_e

u_r_d = str(sym.N(u_r[0],8))
u_r_q = str(sym.N(u_r[1],8))

print(f'Du_r_1 = {u_r_d};')
print(f'Du_r_2 = {u_r_q};')

print('\nWarning: Control output is eta_dq!!')

Du_r_1 = -2715.7977*Dx_i_1 + 1040.9568*Dx_i_2 - 0.071871552*Dx_r_1 + 0.00052148736*Dx_r_2 - 1.8262175*xD[4] + 0.019754652*xD[5] - 0.0048070784*xD[6] + 0.0010494948*xD[7] - 3.4543535*xD[8] + 0.39761572*xD[9];
Du_r_2 = -1040.9568*Dx_i_1 - 2715.7977*Dx_i_2 - 0.00052148736*Dx_r_1 - 0.071871552*Dx_r_2 - 0.019754652*xD[4] - 1.8262175*xD[5] - 0.0010494948*xD[6] - 0.0048070784*xD[7] - 0.39761572*xD[8] - 3.4543535*xD[9];



## Symbolic obtention of the plant model

In [44]:
R_t,L_t,C_m,G_d,R_s,L_s = sym.symbols('R_t,L_t,C_m,G_d,R_s,L_s', real=True)
i_tD,i_tQ,v_mD,v_mQ,i_sD,i_sQ = sym.symbols('i_tD,i_tQ,v_mD,v_mQ,i_sD,i_sQ', real=True)
v_tD,v_tQ = sym.symbols('v_tD,v_tQ', real=True)
v_sD,v_sQ = sym.symbols('v_sD,v_sQ', real=True)

A = sym.Matrix([
    [-R_t/L_t,       0,   -1/L_t,        0,        0,        0],
    [       0,-R_t/L_t,        0,   -1/L_t,        0,        0],
    [   1/C_m,       0, -G_d/C_m,        0,   -1/C_m,        0],
    [       0,   1/C_m,        0, -G_d/C_m,        0,   -1/C_m],
    [       0,       0,    1/L_s,        0, -R_s/L_s,        0],
    [       0,       0,        0,    1/L_s,        0, -R_s/L_s],
    ])

B = sym.Matrix([
    [ 1/L_t,        0],
    [       0,  1/L_t],
    [       0,      0],
    [       0,      0],
    [       0,      0],
    [       0,      0],
    ])

B_g = sym.Matrix([
    [       0,      0],
    [       0,      0],
    [       0,      0],
    [       0,      0],
    [-1/L_s,        0],
    [       0, -1/L_s],
    ])

x = sym.Matrix([i_tD,i_tQ,v_mD,v_mQ,i_sD,i_sQ])
u = sym.Matrix([v_tD,v_tQ])
u_g = sym.Matrix([v_sD,v_sQ])

dx_new = A@x + B@u + B_g@u_g

di_tD = 1/L_t*(v_tD - R_t*i_tD - v_mD)  
di_tQ = 1/L_t*(v_tQ - R_t*i_tQ - v_mQ) 
dv_mD = 1/C_m*(i_tD - G_d*v_mD - i_sD) 
dv_mQ = 1/C_m*(i_tQ - G_d*v_mQ - i_sQ) 
di_sD = 1/L_s*(v_mD - R_s*i_sD - v_sD)  
di_sQ = 1/L_s*(v_mQ - R_s*i_sQ - v_sQ)
dx = sym.Matrix([di_tD,di_tQ,dv_mD,dv_mQ,di_sD,di_sQ])

dx = sym.Matrix([di_tD,di_tQ,dv_mD,dv_mQ,di_sD,di_sQ])
sym.simplify(dx_new - dx)  # just to check the model is ok

Matrix([
[0],
[0],
[0],
[0],
[0],
[0]])